# LLM - PromptTemplate


In [2]:
from langchain import PromptTemplate
from langchain_openai import OpenAI

llm = OpenAI()


# usage1
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

prompt = prompt_template.format(adjective="funny", content="chickens")
print(llm(prompt))


# usage2
prompt_template2 = PromptTemplate(
    input_variables=["adjective", "content"],
    template="Tell me a {adjective} joke about {content}.",
)
prompt2 = prompt_template2.format(adjective="funny", content="brid")
print(llm(prompt2))
print(llm("《沙丘》和《沙丘2》的导演是同一个国家的吗？"))

/Users/liangzhu/anaconda3/envs/llm/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




Why did the chicken go to the seance?

To talk to the other side of the road!


Why did the bridge cross the road?

To get to the other tide!


是的，《沙丘》和《沙丘2》的导演都是来自加拿大的丹尼·维伦纽瓦。


### Few-shot Prompt


In [11]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

examples = [
    {
        "question": "乔治·华盛顿的外祖父是谁？",
        "answer": """
            这里需要进一步的问题吗：是的。
            追问：谁是乔治·华盛顿的母亲？
            中间答案：乔治·华盛顿的母亲是Mary Ball Washington。
            追问：Mary Ball Washington的父亲是谁？
            中间答案：Mary Ball Washington的父亲是Joseph Ball。
            所以最终答案是：Joseph Ball
            """,
    },
    {
        "question": "《大白鲨》和《皇家赌场》的导演是同一个国家的吗？",
        "answer": """
            这里需要进一步的问题吗：是的。
            追问：谁是《大白鲨》的导演？
            中间答案：《大白鲨》的导演是Steven Spielberg。
            追问：Steven Spielberg来自哪里？
            中间答案：美国。
            追问：谁是《皇家赌场》的导演？
            中间答案：《皇家赌场》的导演是Martin Campbell。
            追问：Martin Campbell来自哪里？
            中间答案：新西兰。
            所以最终答案是：不是
            """,
    },
]

example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question：{question}\nAnswer：{answer}",
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(llm(few_shot_prompt.format(input="《沙丘》和《沙丘2》的导演是同一个国家的吗？")))


Answer:
            这里需要进一步的问题吗：是的。
            追问：谁是《沙丘》的导演？
            中间答案：《沙丘》的导演是Denis Villeneuve。
            追问：Denis Villeneuve来自哪里？
            中间答案：加拿大。
            追问：谁是《沙丘2》的导演？
            中间答案：《沙丘2》的导演是Denis Villeneuve。
            追问：Denis Villeneuve来自哪里？
            中间答案：加拿大。
            所以最终答案是：是的。


#### 根据某种条件或规则来自动选择 few prompt


In [13]:
# 从给定的示例中选择最相似的示例来返回输入的反义词
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma(),
    k=1,
)

similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="What is the opposite of input?",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

prompt = similar_prompt.format(adjective="long")
print(llm(prompt))

 short


# Chat Model - ChatPromptTemplate


In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}"),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = template.format_messages(name="Bob", user_input="What is the IPhone?")
print(chat_model(messages))

content="The iPhone is a line of smartphones designed and marketed by Apple Inc. It runs on Apple's iOS operating system and has become one of the most popular smartphones in the world."
